[View in Colaboratory](https://colab.research.google.com/github/Better-Boy/inkers-assignment-solutions/blob/master/Assignment%204B.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 60
num_filter = 24
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l = 15
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)



In [18]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

__________________________________________________________________________________________________
activation_54 (Activation)      (None, 4, 4, 72)     0           batch_normalization_54[0][0]     
__________________________________________________________________________________________________
conv2d_55 (Conv2D)              (None, 4, 4, 12)     7776        activation_54[0][0]              
__________________________________________________________________________________________________
dropout_54 (Dropout)            (None, 4, 4, 12)     0           conv2d_55[0][0]                  
__________________________________________________________________________________________________
concatenate_51 (Concatenate)    (None, 4, 4, 84)     0           concatenate_50[0][0]             
                                                                 dropout_54[0][0]                 
__________________________________________________________________________________________________
batch_norm

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 328s 7ms/step - loss: 1.5318 - acc: 0.4351 - val_loss: 2.8895 - val_acc: 0.2946
Epoch 2/10
19200/50000 [==========>...................] - ETA: 2:57 - loss: 1.1994 - acc: 0.5645

50000/50000 [==============================] - 311s 6ms/step - loss: 1.1280 - acc: 0.5923 - val_loss: 1.7641 - val_acc: 0.4744
Epoch 3/10
45440/50000 [==========================>...] - ETA: 26s - loss: 0.9468 - acc: 0.6621

50000/50000 [==============================] - 310s 6ms/step - loss: 0.9441 - acc: 0.6636 - val_loss: 1.5142 - val_acc: 0.5482
Epoch 4/10
50000/50000 [==============================] - 309s 6ms/step - loss: 0.8329 - acc: 0.7049 - val_loss: 0.9263 - val_acc: 0.6818
Epoch 5/10
 3200/50000 [>.............................] - ETA: 4:28 - loss: 0.7724 - acc: 0.7212

50000/50000 [==============================] - 308s 6ms/step - loss: 0.7608 - acc: 0.7302 - val_loss: 1.0018 - val_acc: 0.6792
Epoch 6/10
39424/50000 [======================>.......] - ETA: 1:00 - loss: 0.7032 - acc: 0.7515

50000/50000 [==============================] - 309s 6ms/step - loss: 0.7006 - acc: 0.7528 - val_loss: 0.9454 - val_acc: 0.7034
Epoch 7/10
50000/50000 [==============================] - 309s 6ms/step - loss: 0.6554 - acc: 0.7707 - val_loss: 1.3383 - val_acc: 0.6431
Epoch 8/10
 1792/50000 [>.............................] - ETA: 4:38 - loss: 0.5985 - acc: 0.7919

50000/50000 [==============================] - 310s 6ms/step - loss: 0.6178 - acc: 0.7825 - val_loss: 0.8128 - val_acc: 0.7407
Epoch 9/10
38784/50000 [======================>.......] - ETA: 1:04 - loss: 0.5813 - acc: 0.7948

50000/50000 [==============================] - 309s 6ms/step - loss: 0.5807 - acc: 0.7947 - val_loss: 0.8614 - val_acc: 0.7351
Epoch 10/10
50000/50000 [==============================] - 309s 6ms/step - loss: 0.5522 - acc: 0.8069 - val_loss: 0.8322 - val_acc: 0.7510


In [0]:
model.save_weights('dnn_10.h5')

In [0]:
model.load_weights('dnn_10.h5')

In [15]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 309s 6ms/step - loss: 0.5285 - acc: 0.8142 - val_loss: 1.0988 - val_acc: 0.6975
Epoch 2/10
19200/50000 [==========>...................] - ETA: 2:57 - loss: 0.4916 - acc: 0.8313

50000/50000 [==============================] - 309s 6ms/step - loss: 0.4971 - acc: 0.8263 - val_loss: 0.6831 - val_acc: 0.7884
Epoch 3/10
45440/50000 [==========================>...] - ETA: 26s - loss: 0.4778 - acc: 0.8330

50000/50000 [==============================] - 310s 6ms/step - loss: 0.4774 - acc: 0.8330 - val_loss: 0.7263 - val_acc: 0.7870
Epoch 4/10
50000/50000 [==============================] - 310s 6ms/step - loss: 0.4597 - acc: 0.8400 - val_loss: 0.9638 - val_acc: 0.7329
Epoch 5/10
 3200/50000 [>.............................] - ETA: 4:29 - loss: 0.4562 - acc: 0.8416

50000/50000 [==============================] - 310s 6ms/step - loss: 0.4466 - acc: 0.8429 - val_loss: 0.6261 - val_acc: 0.8037
Epoch 6/10
39424/50000 [======================>.......] - ETA: 1:00 - loss: 0.4259 - acc: 0.8505

50000/50000 [==============================] - 310s 6ms/step - loss: 0.4265 - acc: 0.8500 - val_loss: 0.9621 - val_acc: 0.7533
Epoch 7/10
50000/50000 [==============================] - 310s 6ms/step - loss: 0.4113 - acc: 0.8562 - val_loss: 0.9122 - val_acc: 0.7588
Epoch 8/10
 1792/50000 [>.............................] - ETA: 4:37 - loss: 0.4041 - acc: 0.8555

50000/50000 [==============================] - 310s 6ms/step - loss: 0.3907 - acc: 0.8631 - val_loss: 0.7369 - val_acc: 0.7895
Epoch 9/10
38784/50000 [======================>.......] - ETA: 1:04 - loss: 0.3807 - acc: 0.8658

50000/50000 [==============================] - 309s 6ms/step - loss: 0.3831 - acc: 0.8647 - val_loss: 0.7293 - val_acc: 0.7945
Epoch 10/10
50000/50000 [==============================] - 309s 6ms/step - loss: 0.3707 - acc: 0.8695 - val_loss: 0.7152 - val_acc: 0.7978


In [0]:
model.save_weights('dnn_20.h5')

In [0]:
model.load_weights('dnn_20.h5')

In [13]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 327s 7ms/step - loss: 0.3655 - acc: 0.8716 - val_loss: 0.7623 - val_acc: 0.7848
Epoch 2/10
19200/50000 [==========>...................] - ETA: 2:56 - loss: 0.3435 - acc: 0.8806

50000/50000 [==============================] - 308s 6ms/step - loss: 0.3485 - acc: 0.8767 - val_loss: 0.8079 - val_acc: 0.7883
Epoch 3/10
45568/50000 [==========================>...] - ETA: 25s - loss: 0.3351 - acc: 0.8816

50000/50000 [==============================] - 308s 6ms/step - loss: 0.3357 - acc: 0.8810 - val_loss: 0.6113 - val_acc: 0.8252
Epoch 4/10
50000/50000 [==============================] - 309s 6ms/step - loss: 0.3271 - acc: 0.8843 - val_loss: 0.7509 - val_acc: 0.8029
Epoch 5/10
 3328/50000 [>.............................] - ETA: 4:27 - loss: 0.3099 - acc: 0.8933

50000/50000 [==============================] - 309s 6ms/step - loss: 0.3183 - acc: 0.8870 - val_loss: 1.8604 - val_acc: 0.6419
Epoch 6/10
39424/50000 [======================>.......] - ETA: 1:00 - loss: 0.3126 - acc: 0.8901

50000/50000 [==============================] - 308s 6ms/step - loss: 0.3081 - acc: 0.8917 - val_loss: 1.0811 - val_acc: 0.7388
Epoch 7/10
50000/50000 [==============================] - 308s 6ms/step - loss: 0.3011 - acc: 0.8932 - val_loss: 0.6555 - val_acc: 0.8234
Epoch 8/10
 1792/50000 [>.............................] - ETA: 4:36 - loss: 0.2574 - acc: 0.9118

50000/50000 [==============================] - 308s 6ms/step - loss: 0.2920 - acc: 0.8956 - val_loss: 0.9518 - val_acc: 0.7822
Epoch 9/10
38784/50000 [======================>.......] - ETA: 1:04 - loss: 0.2793 - acc: 0.9018

50000/50000 [==============================] - 308s 6ms/step - loss: 0.2804 - acc: 0.9006 - val_loss: 0.8129 - val_acc: 0.7945
Epoch 10/10
50000/50000 [==============================] - 309s 6ms/step - loss: 0.2765 - acc: 0.9021 - val_loss: 0.7832 - val_acc: 0.8039


In [0]:
model.save_weights('dnn_30.h5')

In [15]:
!ls

datalab  dnn_10.h5  dnn_20.h5  dnn_30.h5


In [16]:
model.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 29s 3ms/step


[0.7831518373608589, 0.8039]

In [0]:
model.load_weights('dnn_30.h5')

In [22]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 313s 6ms/step - loss: 0.2762 - acc: 0.9030 - val_loss: 0.5655 - val_acc: 0.8453
Epoch 2/5
19456/50000 [==========>...................] - ETA: 2:55 - loss: 0.2486 - acc: 0.9127

50000/50000 [==============================] - 309s 6ms/step - loss: 0.2615 - acc: 0.9083 - val_loss: 0.6254 - val_acc: 0.8320
Epoch 3/5
45824/50000 [==========================>...] - ETA: 24s - loss: 0.2515 - acc: 0.9116

50000/50000 [==============================] - 310s 6ms/step - loss: 0.2521 - acc: 0.9117 - val_loss: 0.6859 - val_acc: 0.8254
Epoch 4/5
50000/50000 [==============================] - 310s 6ms/step - loss: 0.2520 - acc: 0.9117 - val_loss: 0.7781 - val_acc: 0.8090
Epoch 5/5
 3456/50000 [=>............................] - ETA: 4:27 - loss: 0.2135 - acc: 0.9236

50000/50000 [==============================] - 310s 6ms/step - loss: 0.2438 - acc: 0.9119 - val_loss: 0.8992 - val_acc: 0.7941


In [0]:
model.save_weights('dnn_35.h5')

In [0]:
model.load_weights('dnn_35.h5')

In [25]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 310s 6ms/step - loss: 0.2414 - acc: 0.9137 - val_loss: 1.8829 - val_acc: 0.6768
Epoch 2/5
19456/50000 [==========>...................] - ETA: 2:56 - loss: 0.2292 - acc: 0.9175

50000/50000 [==============================] - 310s 6ms/step - loss: 0.2361 - acc: 0.9154 - val_loss: 0.5166 - val_acc: 0.8584
Epoch 3/5
45824/50000 [==========================>...] - ETA: 24s - loss: 0.2227 - acc: 0.9210

50000/50000 [==============================] - 310s 6ms/step - loss: 0.2242 - acc: 0.9206 - val_loss: 0.7665 - val_acc: 0.8180
Epoch 4/5
50000/50000 [==============================] - 311s 6ms/step - loss: 0.2214 - acc: 0.9210 - val_loss: 0.7988 - val_acc: 0.8116
Epoch 5/5
 3456/50000 [=>............................] - ETA: 4:28 - loss: 0.2037 - acc: 0.9334

50000/50000 [==============================] - 310s 6ms/step - loss: 0.2171 - acc: 0.9234 - val_loss: 0.7761 - val_acc: 0.8185


In [0]:
model.save_weights('dnn_40.h5')

In [0]:
model.load_weights('dnn_40.h5')

In [28]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 310s 6ms/step - loss: 0.2124 - acc: 0.9238 - val_loss: 0.6465 - val_acc: 0.8455
Epoch 2/5
19456/50000 [==========>...................] - ETA: 2:55 - loss: 0.1967 - acc: 0.9293

50000/50000 [==============================] - 310s 6ms/step - loss: 0.2027 - acc: 0.9281 - val_loss: 0.5958 - val_acc: 0.8522
Epoch 3/5
45824/50000 [==========================>...] - ETA: 24s - loss: 0.2016 - acc: 0.9277

50000/50000 [==============================] - 310s 6ms/step - loss: 0.2015 - acc: 0.9276 - val_loss: 0.8904 - val_acc: 0.7966
Epoch 4/5
50000/50000 [==============================] - 310s 6ms/step - loss: 0.1929 - acc: 0.9310 - val_loss: 0.6466 - val_acc: 0.8437
Epoch 5/5
 3456/50000 [=>............................] - ETA: 4:28 - loss: 0.1944 - acc: 0.9282

50000/50000 [==============================] - 310s 6ms/step - loss: 0.1937 - acc: 0.9298 - val_loss: 1.1789 - val_acc: 0.7674


In [30]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 310s 6ms/step - loss: 0.1872 - acc: 0.9327 - val_loss: 0.8053 - val_acc: 0.8205
Epoch 2/5
19456/50000 [==========>...................] - ETA: 2:56 - loss: 0.1767 - acc: 0.9366

50000/50000 [==============================] - 310s 6ms/step - loss: 0.1837 - acc: 0.9346 - val_loss: 0.8433 - val_acc: 0.8114
Epoch 3/5
45824/50000 [==========================>...] - ETA: 24s - loss: 0.1840 - acc: 0.9328

50000/50000 [==============================] - 310s 6ms/step - loss: 0.1854 - acc: 0.9321 - val_loss: 0.7707 - val_acc: 0.8264
Epoch 4/5
50000/50000 [==============================] - 311s 6ms/step - loss: 0.1782 - acc: 0.9360 - val_loss: 0.6932 - val_acc: 0.8451
Epoch 5/5
 3456/50000 [=>............................] - ETA: 4:28 - loss: 0.1555 - acc: 0.9436

50000/50000 [==============================] - 310s 6ms/step - loss: 0.1741 - acc: 0.9375 - val_loss: 0.5832 - val_acc: 0.8609


In [31]:
model.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 29s 3ms/step


[0.5831894744038582, 0.8609]

In [0]:
model.save_weights('dnn_50.h5')

In [0]:
from google.colab import files

files.download('dnn_50.h5')